In [ ]:
% matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import coo_matrix, eye
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, Birch, DBSCAN



genres = ["Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

# 20 mln
# ratings = pd.read_csv("data/ml-20m/ratings.csv")
# movies = pd.read_csv("data/ml-20m/movies.csv")

# 1 mln
ratings = pd.read_csv("data/ml-1m/ratings.dat", sep="::", names=["userId", "movieId", "rating", "timestamp"], engine='python')
movies = pd.read_csv("data/ml-1m/movies.dat", sep="::", names=["movieId", "title", "genres"], engine='python')

# Macierz użyteczności
rzędy = użytkownicy, 
kolumy = filmy

Może się okazać, że będzie trzeba zastosować inny rodzaj sparse_matrix

In [ ]:
utility_matrix = coo_matrix((ratings['rating'], (ratings['userId'], ratings['movieId']))).asfptype()

utility_matrix.toarray(), utility_matrix.toarray().shape

# movies.head()
# ratings.head()

# Maicerz podobieństwa

In [ ]:
%%time
A = utility_matrix @ utility_matrix.T
S = np.sqrt((A * eye(A.shape[0])).toarray() @ np.ones(A.shape))
Zero = np.logical_or(np.logical_or(S == 0, S.T == 0), np.eye(S.shape[0]))
S = S + np.logical_or(S == 0, S.T == 0)
similarity = (A.toarray() / S / S.T) * (1-Zero)
similarity.shape

In [ ]:
def movie_values(user, n):
    closest_users = np.argpartition((utility_matrix.toarray() > 0) * -similarity[:,[user]],kth=n, axis=0)[:n,]
    closest_users_raitings = utility_matrix.toarray()[
        closest_users,
        np.arange(utility_matrix.shape[1])[None,:]
    ]
    movie_val = np.sum(closest_users_raitings, axis=0)/n
    return movie_val

#TODO: Wyzerwoać te które już ocenił i potem posortować                 
movie_values(user=1021,n=2)

# Wektory gatunków filmowych

In [ ]:
for i, v in enumerate(genres):
    movies[i] = movies['genres'].str.contains(v)
# tags = pd.read_csv("data/tags.csv")

movies = movies.merge(ratings.groupby('movieId')['rating'].agg([pd.np.mean]), how='left', on='movieId')
movies.head()
# ratings.head()

In [ ]:
ratingsExt = pd.merge(ratings, movies, on='movieId')
ratingsExt.head()

In [ ]:
for i in range(18):
    ratingsExt[i] = ratingsExt[i] * (ratingsExt['rating'] - ratingsExt['mean']) 

ratingsExt.head()


In [ ]:
ratingsExt.groupby('userId')[list(range(18))].agg([pd.np.sum]).head()

In [ ]:
data = ratingsExt.drop(["timestamp", "title", "genres"], axis = 1)
data.head()

## Przykłądowe dane:

In [ ]:
ratingsExt.loc[(ratingsExt['userId'] == 1) & (ratingsExt['rating'] > 4)]

## Próba grupowania filmów według gatunków (w seumie bez sensu)

In [ ]:

test = movies.drop(["movieId", 'genres'], axis = 1)

data = test.loc[:, range(18)]
print(data.shape)

pca_model = PCA(n_components=2)
pca = pca_model.fit_transform(data)

kmeans = KMeans(n_clusters=4, init='k-means++', n_init=100).fit(data)
birch = Birch(threshold=0.2, n_clusters=None).fit(data)
dbscan = DBSCAN(eps=0.999999, min_samples=25).fit(data)

In [ ]:
x, y = pca[:,0], pca[:,1]


plt.axis('equal')
plt.scatter(x,y, c=kmeans.labels_)
plt.show()

plt.axis('equal')
plt.scatter(x,y, c=birch.labels_)
plt.show()...

plt.axis('equal')
plt.scatter(x,y, c=dbscan.labels_)
plt.show()

# Faktoryzacja macierzy
### Na razie na małych danych

In [ ]:
u, s, vh = svds(utility_matrix, k = 1000)
u.shape, s.shape, vh.shape

In [ ]:
mean_squared_error(utility_matrix.toarray()[utility_matrix.nonzero()], (u * s @ vh)[utility_matrix.nonzero()])

In [ ]:
um_arr_non_zero = utility_matrix.nonzero()
um_arr = utility_matrix.toarray()[um_arr_non_zero]

f = open('MSEs.txt','w')

for k in range(3500, 3901, 100):
    u, s, vh = svds(utility_matrix, k = k)
    val = mean_squared_error(um_arr, (u * s @ vh)[um_arr_non_zero])
    np.save(f"svd_table_{k}", u * s @ vh)
    print(f"MSE dla k={k}: {val}")
    f.write(f"MSE dla k={k}: {val}\n")
    
f.close()

In [ ]:
arr = np.load('svd_table_3900.npy')
arr.shape

In [ ]:
arr

In [ ]:
u, s, vh = svds(utility_matrix, k = 10)
u.shape, s.shape, vh.shape

In [ ]:
u * s @ vh